In [1]:
include("../src/Julia.jl")

pcgWorker (generic function with 1 method)

In [2]:
pwd()

"/Users/serbanstan/git/TreePCG/julia/testing"

In [3]:
graphp = "../../graphs/rand_u_1000_3/"

"../../graphs/rand_u_1000_3/"

In [4]:
a = readFromFile(graphp * "graph.mtx");

In [12]:
tree = []
for i in 1:5
    tr = readFromFile(graphp * "tree$(i).mtx");
    push!(tree, tr);
end

In [13]:
x = readFromFile(graphp * "x.vec");

In [14]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [15]:
numIts = 200;

for i in 4:5
    @time F = treeSolver(tree[i])
    @time F(b);
    
    # a run on standard Float64 data types
    @time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
    norm(lap(a) * myx64 - b) / norm(b)
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_tree$(i)_default.txt", dbg)
    
    # now useing higer precision data types
    for hp in [64,128,256,512,1024]
        set_bigfloat_precision(hp)
        laHP = toHighPrecision(la; precision=hp)
        treeHP = toHighPrecision(tree[i]; precision=hp)
        xHP = toHighPrecision(x; precision=hp);
        bHP = toHighPrecision(b; precision=hp);
        FHP = treeSolver(treeHP)

        @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
        insert!(dbg, 1, "$(numIts) iterations")
        insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
        writeToFile(graphp * "_log_julia_tree$(i)_$(hp).txt", dbg)
    end
end

  0.309238 seconds (275.40 k allocations: 11.699 MB, 2.51% gc time)
  0.160966 seconds (123.31 k allocations: 4.950 MB)
  0.783404 seconds (3.17 M allocations: 95.413 MB, 3.07% gc time)
Working on iteration 10 with a-norm error 1595.982596632244
Working on iteration 20 with a-norm error 726.3371015700478
Working on iteration 30 with a-norm error 266.24248960026193
Working on iteration 40 with a-norm error 102.43142009315562
Working on iteration 50 with a-norm error 45.922040250990975
Working on iteration 60 with a-norm error 20.430403891409647
Working on iteration 70 with a-norm error 7.974305870023791
Working on iteration 80 with a-norm error 3.412132342311741
Working on iteration 90 with a-norm error 1.3520848065141846
Working on iteration 100 with a-norm error 0.5739865942397264
Working on iteration 110 with a-norm error 0.2706449105339926
Working on iteration 120 with a-norm error 0.10432898533530598
Working on iteration 130 with a-norm error 0.039288971043797395
Working on iterati